In [5]:
import psycopg2

conn=psycopg2.connect(database="chengdu_taxi", user="checker", password="201205", port="6666")

cursor=conn.cursor()

In [6]:
cursor.execute("select * from dijkstra limit 10")

results=cursor.fetchall()
for row in results:
    print(row)

(104.07437, 30.71442, 104.07437, 30.71442, 0.0, '[(104.07437, 30.71442)]')
(104.07437, 30.71442, 104.07431, 30.71446, 7.260064822757936, '[(104.07437, 30.71442), (104.07431, 30.71446)]')
(104.07437, 30.71442, 104.07436, 30.71431, 12.234063579284895, '[(104.07437, 30.71442), (104.07436, 30.71431)]')
(104.07437, 30.71442, 104.07423, 30.71451, 16.71889678545483, '[(104.07437, 30.71442), (104.07431, 30.71446), (104.07423, 30.71451)]')
(104.07437, 30.71442, 104.07449, 30.71425, 22.078137949920464, '[(104.07437, 30.71442), (104.07449, 30.71425)]')
(104.07437, 30.71442, 104.07418, 30.71443, 34.01426814156872, '[(104.07437, 30.71442), (104.07436, 30.71431), (104.07418, 30.71443)]')
(104.07437, 30.71442, 104.07389, 30.71473, 57.413558164257395, '[(104.07437, 30.71442), (104.07431, 30.71446), (104.07423, 30.71451), (104.07389, 30.71473)]')
(104.07437, 30.71442, 104.07382, 30.71478, 66.11431858902418, '[(104.07437, 30.71442), (104.07431, 30.71446), (104.07423, 30.71451), (104.07389, 30.71473), (1

In [7]:
p1=(104.07437, 30.71440)
p2=(104.07389, 30.71899)

sql="select * from nodes order by abs({}-lng)+abs({}-lat)".format(p1[0], p1[1])
cursor.execute(sql)
start_nodes=cursor.fetchall()

sql="select * from nodes order by abs({}-lng)+abs({}-lat)".format(p2[0], p2[1])
cursor.execute(sql)
end_nodes=cursor.fetchall()

print(start_nodes[:10])
print("-------------------")
print(end_nodes[:10])

[(104.07437, 30.71442), (104.07436, 30.71431), (104.07431, 30.71446), (104.07418, 30.71443), (104.07423, 30.71451), (104.07449, 30.71425), (104.07383, 30.71465), (104.07389, 30.71473), (104.07382, 30.71478), (104.07452, 30.71324)]
-------------------
[(104.07431, 30.71923), (104.07423, 30.71866), (104.07422, 30.71862), (104.0744, 30.7192), (104.07425, 30.71937), (104.07421, 30.71853), (104.0741517, 30.7180852), (104.07435, 30.72), (104.07455, 30.72008), (104.07403, 30.71734)]


In [10]:
row=None

left=0
right=0
step=1
flag=True

# while not row:
#     sql="select distance, path from dijkstra where source_lng={} and source_lat={} and target_lng={} and target_lat={}".format(start_nodes[left][0], start_nodes[left][1], end_nodes[right][0], end_nodes[right][1])

#     cursor.execute(sql)
#     row=cursor.fetchone()

#     if not row:
#         if left+step>len(start_nodes)-1 or right+step>len(end_nodes)-1:
#             left=0
#             right=0
#             step+=1
#             # print(step)
#             flag=False
#         if left==right:
#             right+=step
#         elif right==left+step:
#             left+=step
#             right-=step
#         elif left==right+step:
#             right+=step
#             if not flag:
#                 left-=step-1
#                 right+=1

#         continue

sql="select distance, path from dijkstra order by abs(source_lng-{})+abs(source_lat-{})+abs(target_lng-{})+abs(target_lat-{}) limit 1".format(p1[0], p1[1], p2[0], p2[1])

cursor.execute(sql)
row=cursor.fetchone()

distance=row[0]
node_path=eval(row[1])
print(distance, node_path)

248.68083397029991 [(104.07437, 30.71442), (104.07394, 30.71661)]


In [13]:
import shapely.wkt as wkt
import geopandas as gp

edges=[]
for i in range(len(node_path)-1):
    sql="select edge from nodes_edge where source_lng={} and source_lat={} and target_lng={} and target_lat={}".format(node_path[i][0], node_path[i][1], node_path[i+1][0], node_path[i+1][1])

    cursor.execute(sql)
    row=cursor.fetchone()

    edge=row[0]
    edge=wkt.loads(edge)

    edges.append(edge)

roads=gp.GeoSeries(edges)
roads

0    LINESTRING (104.07437 30.71442, 104.07420 30.7...
dtype: geometry

In [18]:
import numpy as np

a=[1]*100
b=[1]*100
flags=[[0]*100]*100

left=0
right=0
step=1

while True:
    if step>=len(a):
        break

    flags[left][right]=1

    if left+step>len(a)-1 or right+step>len(b)-1:
        left=0
        right=0
        step+=1
        # print(step)
        flag=False
    if left==right:
        right+=step
    elif right==left+step:
        left+=step
        right-=step
    elif left==right+step:
        right+=step
        if not flag:
            left-=step-1
            right+=1

print(np.where(flags==0))

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
(array([], dtype=int64),)
